In [4]:
# importing all the modules 

import sys
import math
import laspy
import pyproj

import numpy as np
import pandas as pd
import geopandas as gpd

from datetime import datetime
from pycrown import PyCrown
from shapely.geometry import Polygon

In [5]:
# Define the input folder where the files are stored
input_folder = 'input'

# Paths to the input files including DSM, DTM, CHM, and LiDAR point cloud,
# already clipped to the study site extent.
F_CHM = f'{input_folder}/clipped_CHM.tif'
F_DTM = f'{input_folder}/clipped_DTM.tif'
F_DSM = f'{input_folder}/clipped_DSM.tif'
F_LAS = f'{input_folder}/clipped_pointcloud.las'

In [6]:
# Initialize the PyCrown class with the provided input files
# F_CHM: Path to the Canopy Height Model (CHM) file in .tif format
# F_DTM: Path to the Digital Terrain Model (DTM) file in .tif format
# F_DSM: Path to the Digital Surface Model (DSM) file in .tif format
# F_LAS: Path to the LiDAR point cloud file in .las format
# outpath: The directory where the output results will be saved
PC = PyCrown(F_CHM, F_DTM, F_DSM, F_LAS, outpath='result')

In [7]:
# Smoothing of CHM while preserving fine details in the CHM
# The method filter_chm applies a smoothing filter to the Canopy Height Model (CHM).
# Parameters:
# 1: The window size in pixels for smoothing. A value of 1 means no smoothing is applied,
#    as each pixel is considered individually without averaging neighboring values.

# ws_in_pixels=True: Specifies that the window size is given in pixels, not in some other unit.
# circular=False: Indicates that a square window is used for filtering instead of a circular one.

PC.filter_chm(1, ws_in_pixels=True, circular=False)

In [8]:
# Tree detection in the CHM using specified parameters
# The method tree_detection identifies tree tops in the Canopy Height Model (CHM).
# Parameters:
# PC.chm: The Canopy Height Model to be analyzed.
# ws=1: A window size of 1x1 pixel, meaning each pixel is considered independently as a potential tree top.
# hmin=1.3: The minimum height threshold for a pixel to be considered as a tree top. 
#   This has been adjusted from 16.0 to 1.3 meters to detect smaller trees.

PC.tree_detection(PC.chm, ws=1, hmin=1.3)

In [9]:
# Delineation of tree crowns using the specified algorithm and thresholds
# The method crown_delineation is used to define the boundaries of tree crowns in the CHM.

# Define parameters for crown delineation
algorithm = 'dalponteCIRC_numba'  # Algorithm used for crown delineation
th_tree = 1.3  # Minimum height threshold for considering a tree crown in meters
th_seed = 0.5  # Seed threshold factor
th_crown = 0.05  # Crown threshold factor
max_crown = 4.0  # Maximum allowable crown radius in meters

# Delineation of tree crowns using the specified algorithm and thresholds
PC.crown_delineation(algorithm=algorithm,
                     th_tree=th_tree,
                     th_seed=th_seed,
                     th_crown=th_crown,
                     max_crown=max_crown)

Tree crowns delineation: 0.004s


In [11]:
# Correcting tree tops after initial detection
# The correct_tree_tops() function refines the positions of detected tree tops.
# This process includes:
# - Removing false positives: tree tops that were incorrectly identified due to noise or low thresholds.
# - Adjusting the position of tree tops to more accurate locations within the tree crowns.
# - Merging closely positioned tree tops that likely represent the same tree.

# This correction step ensures that the final tree top positions are more accurate, 
# leading to better results in subsequent analyses.
PC.correct_tree_tops()

Number of trees: 6021
Tree tops corrected: 2551
Tree tops corrected: 42.36837734595582%
DSM correction: 475
COM correction: 2076


(475, 2076)

In [12]:
# Calculate tree height and elevation for the detected tree tops
PC.get_tree_height_elevation(loc='top')

# Calculate tree height and elevation for the corrected tree tops
PC.get_tree_height_elevation(loc='top_cor')

In [13]:
# Screen out small trees from the detected tree tops
# hmin=0.2: Minimum height threshold for a tree is 0.2 meters.
PC.screen_small_trees(hmin=0.2, loc='top') 

In [14]:
# Convert detected tree crowns to polygon shapes using a raster-based approach
PC.crowns_to_polys_raster()

# Convert detected tree crowns to smoothed polygon shapes
# and optionally store the corresponding LiDAR points for each crown
PC.crowns_to_polys_smooth(store_las=True)

Converting LAS point cloud to shapely points
Converting raster crowns to shapely polygons
Attach LiDAR points to corresponding crowns
Create convex hull around first return points
Classifying point cloud


In [15]:
# Perform quality control on the detected and processed tree crowns
PC.quality_control()

In [16]:
print(f"Number of trees detected: {len(PC.trees)}")

Number of trees detected: 6021


In [17]:
# Export the Canopy Height Model (CHM) as a raster file
PC.export_raster(PC.chm, PC.outpath / 'chm.tif', 'CHM')

# Export the locations of the initially detected tree tops
PC.export_tree_locations(loc='top')

# Export the locations of the corrected tree tops
PC.export_tree_locations(loc='top_cor')

# Export the tree crowns as raster-based polygons
PC.export_tree_crowns(crowntype='crown_poly_raster')

# Export the tree crowns as smoothed polygons
PC.export_tree_crowns(crowntype='crown_poly_smooth')

Exported 6021 valid tree crowns to result/tree_crown_poly_raster.shp
Skipping unsupported geometry type at index 3
Skipping unsupported geometry type at index 5
Skipping unsupported geometry type at index 6
Skipping unsupported geometry type at index 7
Skipping unsupported geometry type at index 11
Skipping unsupported geometry type at index 12
Skipping unsupported geometry type at index 13
Skipping unsupported geometry type at index 14
Skipping LineString with insufficient points at index 15
Skipping unsupported geometry type at index 16
Skipping unsupported geometry type at index 18
Skipping unsupported geometry type at index 19
Skipping unsupported geometry type at index 20
Skipping unsupported geometry type at index 21
Skipping unsupported geometry type at index 22
Skipping unsupported geometry type at index 23
Skipping unsupported geometry type at index 24
Skipping unsupported geometry type at index 25
Skipping unsupported geometry type at index 26
Skipping unsupported geometry ty

In [103]:


# Convert your DataFrame to a GeoDataFrame using the 'top_cor' column for geometry
# 'top_cor' is assumed to be a column containing geometries (e.g., points representing tree tops)
trees_gdf = gpd.GeoDataFrame(PC.trees, geometry='top_cor')

# Setting the Coordinate Reference System (CRS) for the GeoDataFrame
# EPSG:27700 corresponds to OSGB36 / British National Grid, which is commonly used in the UK
epsg_code = "EPSG:27700"
trees_gdf = trees_gdf.set_crs(epsg_code)

# Function to calculate the diameter of a tree crown polygon
def polygon_diameter(polygon):
    if isinstance(polygon, Polygon):
        # Extract the bounding box of the polygon (minx, miny, maxx, maxy)
        minx, miny, maxx, maxy = polygon.bounds
        # Calculate the diameter as the maximum of the width or height of the bounding box
        diameter = max(maxx - minx, maxy - miny)
        return diameter
    else:
        return None

# Calculate the diameter for each tree crown polygon
# 'crown_poly_raster' is assumed to be a column containing polygon geometries for tree crowns
trees_gdf['diameter'] = trees_gdf['crown_poly_raster'].apply(polygon_diameter)

# Create a DataFrame with tree number, height, and calculated diameter
trees_gdf['tree_number'] = trees_gdf.index  # Assign a unique tree number based on the index
tree_database = trees_gdf[['tree_number', 'top_height', 'diameter']].copy()

# Add the geometry column back to the new DataFrame to make it a GeoDataFrame
tree_database = gpd.GeoDataFrame(tree_database, geometry=trees_gdf.geometry)

In [62]:
# print(tree_database.head(10))

   tree_number  top_height  diameter                       geometry
0            0    1.483002       3.0  POINT (507631.500 246086.500)
1            1    1.958000       3.0  POINT (507629.500 246084.500)
2            2    2.009003       3.0  POINT (507627.500 246083.500)
3            3    1.918003       1.0  POINT (507626.500 246082.500)
4            4    2.667002       5.0  POINT (507623.500 246080.500)
5            5    2.211002       1.0  POINT (507619.500 246079.500)
6            6    1.929001       1.0  POINT (507618.500 246078.500)
7            7    1.359001       2.0  POINT (507597.500 246062.500)
8            8    3.320002       3.0  POINT (507485.500 245984.500)
9            9    3.936003       3.0  POINT (507484.500 245983.500)


In [70]:
# len(tree_database) # ensure that the number is the same as previously 

6021

In [93]:
import geopandas as gpd

# Load the shapefile of the planting areas
planting_areas_gdf = gpd.read_file('SS/Planting Types.shp')

# Perform a spatial join to keep only trees within the planting areas
# 'how="inner"' keeps only the rows where the spatial join condition ('within') is true
trees_within_planting_areas = gpd.sjoin(tree_database, planting_areas_gdf, how='inner', op='within')

# Remove the 'index_right' and 'id' columns if present
# This cleans up the GeoDataFrame by removing unnecessary columns resulting from the join
columns_to_drop = ['index_right', 'id']
trees_within_planting_areas = trees_within_planting_areas.drop(columns=[col for col in columns_to_drop if col in trees_within_planting_areas.columns])

# Load the shapefile containing information about planting years
planting_years_gdf = gpd.read_file('SS/Shocott Spring Area.shp')

# Perform a spatial join to add attributes from the planting years shapefile
# Again, 'how="inner"' ensures that only trees within the specified areas are kept
trees_within_both_areas = gpd.sjoin(trees_within_planting_areas, planting_years_gdf, how='inner', op='within')

# Clean up by removing any redundant columns from the second spatial join
columns_to_drop_second = ['index_right']  # Adjust based on actual column names resulting from the second join
trees_within_both_areas = trees_within_both_areas.drop(columns=[col for col in columns_to_drop_second if col in trees_within_both_areas.columns])

# Rename columns to provide meaningful names
# In this case, renaming 'id' (which might represent a planting year) to 'Year'
trees_within_both_areas = trees_within_both_areas.rename(columns={'id': 'Year'})

In [104]:
#len(trees_within_both_areas)
print(trees_within_both_areas.head(10))

    tree_number  top_height  diameter                       geometry  \
12           12    1.424002       1.0  POINT (507531.000 245983.000)   
16           16    1.940001       1.0  POINT (507489.000 245960.000)   
18           18    2.199003       1.0  POINT (507581.000 245946.000)   
19           19    1.453003       1.0  POINT (507560.000 245938.000)   
20           20    2.310001       1.0  POINT (507611.000 245938.000)   
21           21    1.770002       1.0  POINT (507499.000 245930.000)   
22           22    1.814003       1.0  POINT (507480.000 245914.000)   
23           23    1.422001       1.0  POINT (507492.000 245911.000)   
24           24    1.467001       1.0  POINT (507543.000 245908.000)   
25           25    1.815001       1.0  POINT (507496.000 245890.000)   

          Type  year  
12  Shrub-rich     3  
16  Shrub-rich     3  
18  Shrub-rich     3  
19  Shrub-rich     3  
20  Shrub-rich     3  
21  Shrub-rich     3  
22  Shrub-rich     3  
23  Shrub-rich     3  


In [97]:
# Create a CSV file for further analysis
trees_within_both_areas.to_csv('trees_within_both_areas.csv', index=False)

# Below section is based on Jucker et al. 2017 ("Allometric equations for integrating remote sensing imagery into forest monitoring programmes")

In [98]:
# Reading the CSV file into a DataFrame
df = pd.read_csv('trees_within_both_areas.csv')

In [106]:
# Parameters for DBH calculation
exp_factor_d = np.exp(0.056**2 / 2)

# Calculate DBH (Diameter at Breast Height)
df['DBH'] = 0.557 * (df['top_height'] * df['diameter'])**0.809 * exp_factor_d

In [108]:
# Parameters for AGB (Aboveground Biomass) calculation
exp_factor_agb = np.exp(0.204**2 / 2)

# As there is no option to classify the species of individual trees, 
# it will be assumed that each tree has an equal probability (50% chance) of being either an angiosperm or a gymnosperm.
# A weighted average will then be applied.

# Calculate AGB (Aboveground Biomass) for angiosperms
df['AGB_angiosperm'] = 0.016 * (df['top_height'] * df['diameter'])**2.013 * exp_factor_agb

# Calculate AGB (Aboveground Biomass) for gymnosperms
df['AGB_gymnosperm'] = 0.109 * (df['top_height'] * df['diameter'])**1.790 * exp_factor_agb


In [114]:
# Define the species mix proportions
species_mix_proportions = {
    "Mixed Wood": (0.4, 0.6),  # 40% Angiosperms, 60% Gymnosperms
    "Native Bro": (1.0, 0.0),  # 100% Angiosperms
    "Native Shr": (1.0, 0.0),  # 100% Angiosperms
    "Native Tim": (1.0, 0.0),  # 100% Angiosperms
    "Shrub-rich": (1.0, 0.0)   # 100% Angiosperms
}

# Function to calculate the total AGB for each tree based on the species mix
def calculate_agb_weighted(row):
    mix = row['Type']
    angiosperm_weight, gymnosperm_weight = species_mix_proportions.get(mix, (0, 0))
    return angiosperm_weight * row['AGB_angiosperm'] + gymnosperm_weight * row['AGB_gymnosperm']

# Apply the function to calculate the total AGB and store it in a new column 'AGB_total'
df['AGB_total'] = df.apply(calculate_agb_weighted, axis=1)

In [121]:
print(df.head)

<bound method NDFrame.head of       tree_number  top_height  diameter                   geometry  \
0              12    1.424002       1.0      POINT (507531 245983)   
1              16    1.940001       1.0      POINT (507489 245960)   
2              18    2.199003       1.0      POINT (507581 245946)   
3              19    1.453003       1.0      POINT (507560 245938)   
4              20    2.310001       1.0      POINT (507611 245938)   
...           ...         ...       ...                        ...   
5287         5817    3.610003       5.0  POINT (507902.5 245083.5)   
5288         5866    1.633001       3.0      POINT (507868 245076)   
5289         5889    1.717001       3.0      POINT (507881 245073)   
5290         5964    2.130003       1.0  POINT (507875.5 245057.5)   
5291         6002    2.775002       2.0  POINT (507881.5 245045.5)   

            Type  year       DBH  AGB_angiosperm  AGB_gymnosperm  \
0     Shrub-rich     3  0.742551        0.033279        0.209

In [117]:
# Group the data by species mix ('Type') and calculate the sum of AGB and the number of trees
summary = df.groupby('Type').agg(
    total_AGB=('AGB_total', 'sum'),         # Sum of the total AGB for each species mix type
    number_of_trees=('tree_number', 'count') # Count of trees for each species mix type
).reset_index()

# Display the summary DataFrame
print(summary)

         Type     total_AGB  number_of_trees
0  Mixed Wood  35782.382699             4773
1  Native Bro    199.044372              269
2  Native Shr      8.723947               15
3  Native Tim     68.306814              184
4  Shrub-rich     24.464389               51


In [ ]:
# Save the summary DataFrame to a CSV file
summary.to_csv('AGB_summary_by_species_mix.csv', index=False)i